In [1]:
import torch
# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

from botorch.test_functions import Ackley

# Test the 4D Ackley function
dim = 4
ackley_function = Ackley(dim=dim)

# Define the objective function
def objective_function(X):
    return -ackley_function(2*X-1)

In [2]:
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])
seed = 12
maximize = True
num_iterations = 16

In [3]:
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer
from botorch.utils.sampling import draw_sobol_samples

In [4]:
from botorch.acquisition.predictive_entropy_search import qPredictiveEntropySearch
torch.manual_seed(seed)

init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

# Create an instance of BayesianOptimizer
Optimizer = BayesianOptimizer(objective=objective_function, dim=dim, maximize=maximize, initial_points=init_x, input_standardize=True)
# Run the optimization
Optimizer.run(num_iterations=num_iterations, acquisition_function_class=qPredictiveEntropySearch)

Iteration 0, New point: [0.39110506 0.20300196 0.38761725 0.09529869], New value: -3.7411337971936054
Best observed value: -3.1664714345236615
Current acquisition value: 0.20410020161874387
Cumulative cost: 1.0
Running time: 19.117409

Iteration 1, New point: [0.37384178 0.74328204 0.72511162 0.52581064], New value: -3.315279092166524
Best observed value: -3.1664714345236615
Current acquisition value: 0.3356914665660736
Cumulative cost: 2.0
Running time: 18.268083999999998

Iteration 2, New point: [0.36474371 0.51535933 0.47768493 0.78846777], New value: -2.6960008027854774
Best observed value: -2.6960008027854774
Current acquisition value: 0.2881293282510491
Cumulative cost: 3.0
Running time: 28.850878

Iteration 3, New point: [0.39726052 0.83029592 0.46602561 1.        ], New value: -3.499581849513039
Best observed value: -2.6960008027854774
Current acquisition value: 0.3220318084048206
Cumulative cost: 4.0
Running time: 35.956473

Iteration 4, New point: [0.36033178 0.59217394 0.462

RuntimeError: 20 elements of the 20 element gradient array `gradf` are NaN. This often indicates numerical issues.

In [19]:
Optimizer.x

tensor([[0.2053, 0.3585, 0.6309, 0.5492],
        [0.0285, 0.2932, 0.2715, 0.6113],
        [0.8737, 0.7220, 0.2637, 0.8085],
        [0.6277, 0.0971, 0.3706, 0.8777],
        [0.2851, 0.7546, 0.3960, 0.5146],
        [0.0558, 0.9609, 0.2118, 0.1557],
        [0.0385, 0.7806, 0.3743, 0.7727],
        [0.0129, 0.9296, 0.2388, 0.9026],
        [0.0034, 0.4669, 0.6417, 0.1286],
        [0.4833, 0.7988, 0.2576, 0.5893],
        [0.3911, 0.2030, 0.3876, 0.0953],
        [0.3738, 0.7433, 0.7251, 0.5258],
        [0.3647, 0.5154, 0.4777, 0.7885],
        [0.3973, 0.8303, 0.4660, 1.0000],
        [0.3603, 0.5922, 0.4621, 0.8021],
        [0.3372, 0.5004, 0.4146, 0.4881],
        [0.2911, 0.4964, 0.4377, 0.5077],
        [0.3585, 0.5241, 0.5554, 0.3447],
        [0.4209, 0.4856, 0.6937, 0.6229],
        [0.4805, 0.3491, 0.0000, 0.5672]])

In [20]:
Optimizer.y

tensor([-3.1665, -4.0511, -4.4084, -3.8389, -3.2199, -4.5132, -4.2666, -4.4265,
        -3.5960, -3.3342, -3.7411, -3.3153, -2.6960, -3.4996, -2.9793, -1.7895,
        -1.9907, -2.2431, -2.4476, -2.9350])

In [39]:
from pandora_bayesopt.utils import fit_gp_model
model = fit_gp_model(
        X=Optimizer.x, 
        objective_X=Optimizer.y,
        cost_X=Optimizer.c,
        input_standardize=True,
        unknown_cost=False
    )
model

SingleTaskGP(
  (likelihood): FixedNoiseGaussianLikelihood(
    (noise_covar): FixedGaussianNoise()
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): MaternKernel(
      (lengthscale_prior): GammaPrior()
      (raw_lengthscale_constraint): Positive()
    )
    (outputscale_prior): GammaPrior()
    (raw_outputscale_constraint): Positive()
  )
  (outcome_transform): Standardize()
)

In [40]:
from botorch.sampling.pathwise import draw_matheron_paths
from botorch.utils.sampling import optimize_posterior_samples

# Draw sample path(s)
paths = draw_matheron_paths(model, sample_shape=torch.Size([1]))

# Optimize
optimal_input, _ = optimize_posterior_samples(paths=paths, bounds=torch.stack([torch.zeros(dim), torch.ones(dim)]), maximize=True)

In [41]:
optimal_input

tensor([[0.3624, 0.4551, 0.4685, 0.4943]], grad_fn=<ViewBackward0>)

In [42]:
from botorch.optim import optimize_acqf
PES = qPredictiveEntropySearch(model=model, optimal_inputs=optimal_input, maximize=maximize)
new_point, new_point_PES = optimize_acqf(
        acq_function=PES,
        bounds=bounds,
        q=1,
        num_restarts=10*dim,
        raw_samples=200*dim,
        options={
                "batch_limit": 5,
                "maxiter": 200,
                "method": "L-BFGS-B",
            },
    )

RuntimeError: 20 elements of the 20 element gradient array `gradf` are NaN. This often indicates numerical issues.

In [ ]:
new_point, new_point_PES

(tensor([[0.3473, 0.4760, 0.4747, 0.4838]]), tensor(0.3532))